## pandas與模型間的介面

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [ ]:
data = pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,4.10,1.3
4,5,0.00,-2.0


In [ ]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  ,  4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [ ]:
df2 = pd.DataFrame(data.values,columns=['one','two','three'])#利用values屬性,重新建立columns欄名稱
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,4.10,1.3
4,5.0,0.00,-2.0


In [ ]:
df3 = data.copy()
df3['strings']=['a','b','c','d','e']#新增欄資料
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,4.10,1.3,d
4,5,0.00,-2.0,e


In [ ]:
model_cols =['x0','x1']
data.loc[:,model_cols].values#察看某一欄位

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  ,  4.1 ],
       [ 5.  ,  0.  ]])

In [ ]:
data['category']=pd.Categorical(['a','b','a','a','b'],categories=['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,4.10,1.3,a
4,5,0.00,-2.0,b


In [ ]:
dummies = pd.get_dummies(data.category,prefix='category')
date_with_dummies = data.drop('category',axis=1).join(dummies)
date_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# 利用Pasty建立模型敘述
* [Patsy](https://patsy.readthedocs.io)是一個Python函式庫
  * 一種短字串組成的"公式語法"formula syntas 
  * Pasty對statsmodels中的指定線性模型支援良好
  如`y~x0+x1`

In [ ]:
data =pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
import patsy

In [ ]:
y,x = patsy.dmatrices('y~x0+x1',data)

In [ ]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [ ]:
x

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [ ]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [ ]:
np.asarray(x)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [ ]:
patsy.dmatrices('y~x0+x1+0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [ ]:
coef,resid,_,_ = np.linalg.lstsq(x,y)

<ipython-input-27-7ec417926207>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef,resid,_,_ = np.linalg.lstsq(x,y)


In [ ]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [ ]:
coef = pd.Series(coef.squeeze(),index=x.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## 在Pasty公式中做資料轉換

In [ ]:
y,x = patsy.dmatrices('y~x0+np.log(np.abs(x1)+1)',data)#取log
x

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [ ]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [ ]:
y,x = patsy.dmatrices('y~standardize(x0)+center(x1)',data)#標準化和中心化(格外小心,有狀態轉換)
x

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [ ]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [ ]:
new_data = pd.DataFrame({
    'x0':[6,7,8,9],
    'x1':[3.1,-0.5,0,2.3],
    'y':[1,2,3,4]
})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [ ]:
y,x = patsy.dmatrices('y~I(x0+x1)',data)#兩變數相加
x

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

## 分類表示資料Patsy

In [5]:
data = pd.DataFrame({
    'key1':['a','a','b','b','a','b','a','b'],
    'key2':[0,1,0,1,0,1,0,0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
})

In [ ]:
y,x = patsy.dmatrices('v2~key1',data)
x

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [ ]:
y,x = patsy.dmatrices('v2~key1+0',data)#省略截距時
x

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [ ]:
y,x = patsy.dmatrices('v2~C(key2)',data)#C()可以知道被視為類別
x

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [ ]:
data['key2']=data['key2'].map({0:'zero',1:'one'})
data#可以使用key1 key2互動的term型式

,key1,key2,v1,v2
0,a,NaN,1,-1.0
1,a,NaN,2,0.0
2,b,NaN,3,2.5
3,b,NaN,4,-0.5
4,a,NaN,5,4.0
5,b,NaN,6,-1.2
6,a,NaN,7,0.2
7,b,NaN,8,-1.7


In [6]:
y,x = patsy.dmatrices('v2~key1+key2',data)
x

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2
          1          0     0
          1          0     1
          1          1     0
          1          1     1
          1          0     0
          1          1     1
          1          0     0
          1          1     0
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [7]:
y,x = patsy.dmatrices('v2~key1+key2+key1:key2',data)
x

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2  key1[T.b]:key2
          1          0     0               0
          1          0     1               0
          1          1     0               0
          1          1     1               1
          1          0     0               0
          1          1     1               1
          1          0     0               0
          1          1     0               0
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# statsmodels簡介
* 線性模型、GLM和穩健robust線性模型
* 線性混合效果模型linear muxed effects model
* 變異數分析ANOVA
* 時間序列和狀態空間模型
* 廣義矩估計eneralized ethod of moments

## 計算線性模型

In [8]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf

In [9]:
def dnorm(mean,variance,size=1):
  if isinstance(size,int):
    size=size,
  return mean + np.sqrt(variance) * np.random.randn(*size)

In [10]:
np.random.seed(12345)

N=100
x = np.c_[dnorm(0,0.4,size=N),
      dnorm(0,0.6,size=N),
      dnorm(0,0.2,size=N)]
eps = dnorm(0,0.1,size=N)
beta = [0.1,0.3,0.5]

y = np.dot(x,beta)+eps

In [11]:
x[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [12]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [14]:
x_model = sm.add_constant(x)
x_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [15]:
model = sm.OLS(y,x)

In [16]:
result = model.fit()
result.params

array([0.17826108, 0.22303962, 0.50095093])

In [18]:
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Sat, 04 Feb 2023   Prob (F-statistic):                    7.44e-12
Time:                        02:53:57   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [19]:
data = pd.DataFrame(x,columns=['col0','col1','col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [22]:
results = smf.ols('y~col0 + col1 + col2',data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [23]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [24]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

## 計算時間序列
* 自迴歸程序 autogressive process
* 卡爾曼濾波 Kalman filterng
* 狀態空間模型 state space model
* 多變數自迴歸模型 multi-variate autogressive model

In [26]:
init_x = 4

import random
values = [init_x,init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0,0.1,N)
for i in range(N):
  new_x = values[-1] * b0 + values[-2]*b1+noise[i]
  values.append(new_x)

In [28]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

In [30]:
results.params

array([ 0.00791554,  0.71074082, -0.31689896, -0.06724519,  0.0041215 ,
       -0.00079061])

## scikit-learn 簡介
* 有關機器學習的工具→→[網站連結](https://scikit-learn.org)
* 內含許多非監督式學習和監督式學習


In [32]:
#修改工作路徑
import os
from google.colab import drive
drive.mount('/content/drive')

path="/content/drive/MyDrive/python"
os.chdir(path)
os.listdir()#判斷裡面內容

Mounted at /content/drive


['Python course 02-常數與變數_exercise_answer.pptx',
 'Python course 03-運算式_exercise.pptx',
 '無標題文件.gdoc',
 'Python course 04-流程控制- 判斷_exercises.pptx',
 'Python練習.zip',
 'dataset',
 'ipython_script_test.ipynb',
 'some_module.ipynb',
 'tmp.txt',
 'tmp1.txt',
 'tmp2.txt',
 'ex1.csv',
 'ex2.csv',
 'csv_mindex.csv',
 'ex3.txt',
 'ex4.csv',
 'ex5.csv',
 'tseries.csv',
 'ex7.csv',
 'example.json',
 'banklist.html',
 'frame_pickle',
 'my_data.h5',
 'ex1.xlsx',
 'ex2.xlsx',
 'mydata.sqlite',
 'mytdata.h5']

In [68]:
train = pd.read_csv('dataset/titanic_train.csv')
test = pd.read_csv('dataset/titanic_test.csv')
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [69]:
# 查看是否有遺失值
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [70]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [71]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [72]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [73]:
predictors =['Pclass','IsFemale','Age']
x_train = train[predictors].values
x_test = test[predictors].values
y_train = train['Survived'].values
x_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [74]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [75]:
##利用邏輯斯迴歸
from sklearn.linear_model import LogisticRegression

In [76]:
model = LogisticRegression()

In [77]:
model.fit(x_train,y_train)

LogisticRegression()

In [78]:
y_predict = model.predict(x_test)

In [79]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [80]:
#(y_true == y_predict).mean() 可以利用混淆矩陣 查看正確率
x_train

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       ...,
       [ 3.,  1., 28.],
       [ 1.,  0., 26.],
       [ 3.,  0., 32.]])

In [81]:
from sklearn.linear_model import LogisticRegressionCV
#model_cv = LogisticRegressionCV(10)
#model_cv.fit(x_train,y_train)
from sklearn.model_selection import cross_val_score
medel =LogisticRegression(C=10)
scores = cross_val_score(model,x_train,y_train,cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])

# 繼續你的學習之路
* *lntroduction to Machine Learning with Python*, 作者: **Andress Muller及Sarah Guido**
* *Python Data Science Handbook*, 作者: **Jake VanderPlas(O'Reilly出版)**
* *Python Machine Learning*, 作者:**Sebastian Raschka(Packt出版)**
* *Hands-On Machine Learing with Scikit-learn and TensorFlow*